# Differential Equations in BCMB

# Differential equations and rates

Biochemistry has _many_ differential equations.  Let's consider a zeroth-order reaction:

$\frac{d[C]}{dt} = -k$

Gives us the equation - where the rate at which a drug (or other substance) decays $\frac{d[C]}{dt}$ is dependent only on the constant $k$.  An example of a zeroth order reaction is the oxidation of ethanol in the human liver to acetaldehyde, catalyzed by alcohol dehydrogenase.  As long as there is a high enough amount of alcohol available, the reaction is limited only by how fast the enzyme can work and the amount of enzyme available. 

If we integrate this, we can see that the concentration should behave like:

$C(t) = C_0 - kt$

With $C_0$ the initial amount of alcohol and $t$ as time. The resulting curve looks like this:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
t=np.linspace(0,100, 101) #Here's my time values from 0 to 100
k=5 #rate constant
C_0=1000 #inital amount of alcohol
C=C_0 - k*t

In [ ]:
C

In [ ]:
plt.plot(t, C)

Ok - so let's now consider a "first-order" rate equation, like either _exponential decay_ or _exponential growth_.

These rates generally follow:

$\frac{d[C]}{dt} = -k[C]$

Gives us the equation - where the rate at which a drug (or other substance) decays $\frac{d[C]}{dt}$ is dependent on the amount of the substance $[C]$ times a constant $k$. But . . . how do we solve this?  The rate of _change_ of the concentration is dependent _itself_ on the concentration.  Of course, if you've taken differential equations, you might know that the answer is:

$C(t)=C_0e^{-kt}$

But let's say we _don't_ know that.  What's another way to solve this?

We can use Euler's method (<https://en.wikipedia.org/wiki/Euler_method>).  This is a numerical method where you basically take _tiny_ little baby steps, estimating the rate at each time based on the concentration at that time, recalculating at each point.  Let me illustrate - let's start with an initial concentration of $C_0=1000$ and $k=0.05$.

In [ ]:
C=np.array([1000])
t=np.array([0])
k=0.05
plt.plot(t, C, '-o')

Now let's calculate $C$ at $t = 1$.  First we calculate the rate at $t=0$:

In [ ]:
rate=np.array([-k*C[0]])
rate

So the rate is $-50$ - let's now apply that rate from $t=0$ to calculate the concentration at $t=1$

In [ ]:
dt = 1 # The size of our time step is 1
t=np.append(t, t[0]+dt)
C=np.append(C, C[0]+dt*rate[0])
plt.plot(t, C, '-o')

So now at $t=1$ we have $C=950$. Let's repeat this now for $t=1$

In [ ]:
rate=np.append(rate, -k*C[1])
rate

Ok - the rate is now $-47.5$.  Let's apply this.

In [ ]:
t=np.append(t, t[1]+dt)
C=np.append(C, C[1]+dt*rate[1])
plt.plot(t, C, '-o')

I see a pattern forming.  I bet I can do this with a for loop:

In [ ]:
#Restart my values
C=np.array([1000])
t=np.array([0])
k=0.05
rate=np.array([-k*C[0]])
dt=1
num_points=int(100/dt)

In [ ]:
for i in range(num_points):
    t=np.append(t, t[i]+dt)
    C=np.append(C, C[i]+dt*rate[i-1])
    rate=np.append(rate, -k*C[i])

In [ ]:
plt.plot(t, C, '-o')

And let's compare this _numerical_ solution to the _analytical_ solution

In [ ]:
(-k*t)

In [ ]:
C_analytical=C[0]*np.exp(-k*t)

plt.plot(t, C, 'o')
plt.plot(t, C_analytical, 'k-')

It's . . . . not perfect, but it's pretty close! The reason the curve is inaccurate is because the rate _changes_ through that first $dt$, that first time interval.  If we took smaller intervals, it would get more accurate, but then it would also require more computation.  Let's see what happens with $dt=0.1$ and $dt=10$

For $dt=0.1$

In [ ]:
#Restart my values
C_small=np.array([1000])
t_small=np.array([0])
k=0.05
rate_small=np.array([-k*C_small[0]])
dt_small=0.1
num_points=int(100/dt_small)

In [ ]:
for i in range(num_points):
    t_small=np.append(t_small, t_small[i]+dt_small)
    C_small=np.append(C_small, C_small[i]+dt_small*rate_small[i-1])
    rate_small=np.append(rate_small, -k*C_small[i])

For $dt = 10$

In [ ]:
#Restart my values
C_big=np.array([1000])
t_big=np.array([0])
k=0.05
rate_big=np.array([-k*C_big[0]])
dt_big=10
num_points=int(100/dt_big)

In [ ]:
for i in range(num_points):
    t_big=np.append(t_big, t_big[i]+dt_big)
    C_big=np.append(C_big, C_big[i]+dt_big*rate_big[i-1])
    rate_big=np.append(rate_big, -k*C_big[i])

Now let's plot all 4 lines

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(t_big, C_big, 'r-')
plt.plot(t, C, 'b-')
plt.plot(t_small, C_small, 'g-')
plt.plot(t, C_analytical, 'k-')


Ok - so if we take _too big_ a time step - it becomes illogical - going to less than zero concentration.  If we take a smaller time step, it has a better fit to the _true_ analytical solution.  So you always want to take the smallest time step that is logical.

All this said, it turns out . . . python can help you with this.  You don't have to make these for loops and solve it yourself each time!  Somewhat unsurprisingly, there are tools for it.   
We are going to first use `scipy`'s `integrate` package - specifically the `solve_ivp` command. Some documentation is here (<https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html#ordinary-differential-equations-odeint>) and here (<https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp>)

In [ ]:
from scipy.integrate import solve_ivp

Let's set up for our same equation as before:

$\frac{d[C]}{dt} = -k[C]$

We have to tell solve_ivp about this equation.  We'll do this by defining the function like so:

In [ ]:
def decay(t, C):
    k = 0.05
    rate = -k * C
    return rate

Where `decay` is just the name of the function (like the name of any variable) and `(t, C)` tells you what information the function is getting passed, in this case **t**ime and **C**oncentration.

Now let's run the ode solver, using the same time range for our analytical solution above:

In [ ]:
C_0=[1000] #initial Condition
tspan = [0, 100] #Time span
C_solver = solve_ivp(decay, tspan, C_0)
C_solver

So what does this output?  It looks like it tells us the times it actually used - notice that the times it used _do not_ have regular spacing - the `solve_ivp` solver uses a more sophisticated version of Euler's method (Runge-kutta 4-5) and an _adaptive_ step size.  Without getting into the math, it chooses the step size to minimize the error _and_ the number of steps it needs to take.

We _can_ force it to output values at certain points if we want like so:

In [ ]:
C_0=[1000] #initial Condition
tspan = [0, 100] #Time span
output_times = np.linspace(0,100,101)
C_solver = solve_ivp(decay, tspan, C_0, t_eval=output_times)
C_solver

Let's plot our original loop plot, the analytical plot, and this `solve_ivp` plot:

In [ ]:
C_solver.y

In [ ]:
plt.plot(t, C, 'b-')
plt.plot(t, C_analytical, 'g-')
plt.plot(C_solver.t, C_solver.y[0], 'r-')
plt.legend(['for loop', 'analytical', 'solveivp'])

We can see that the analytical line and the `solve_ivp` line line up nearly perfectly. You likely only see the red line since it is overlapping the green line. You can remove the analytical line plot to verify.   So, we can use solve_ivp to define these functions.

In [ ]:
def nonconstant_decay(t, C):
    if t < 20:
        k=0
    elif t < 60: 
        k=0.05
    else:
        k=0.25
    return -k*C

In [ ]:
C_0=[1000] #initial Condition
tspan = [0, 100] #Time span
output_times = np.linspace(0,100,1001)
C_solver = solve_ivp(nonconstant_decay, tspan, C_0, t_eval=output_times, max_step=1.0)
C_solver

In [ ]:
C_solver.y[0]

In [ ]:
plt.plot(C_solver.t, C_solver.y[0], 'r-')


In [ ]:
plt.plot(C_solver.t, C_solver.y[0], 'r-')
plt.xlim(18, 20)
plt.ylim(999,1002)

## Exercise

Say you have 100 bacteria in an essentially infinite amount of growth media.  Solve the ODE for exponential growth, assuming that the rate of growth of bacteria is given by:

Growth rate $= dB$ where $d$ is a rate constant and $B$ the number of bacteria. Solve for $d$ of 1.39, .693 and .347.  (Hint: Use `args` in `solve_ivp` to make your life easier)

What is the doubling time of bacteria for each of these? To get this you can A) use the plot to estimate B) use the `events` argument to get the solver to give you the answer for the first doubling.

In [ ]:
def bac_growth(t, B, d=1.39):
    # CODE
    return rate

In [ ]:
def doubled(t,B, d=1.39):
    # CODE
    return doubled

In [ ]:
B_0=[100] #initial Condition
tspan = [0, 100] #Time span

# CODE